# 1. Set logging

In [1]:
import logging
reload(logging)
logging.basicConfig(
    format='%(asctime)-9s %(levelname)-8s %(message)s',
    date='%I:%M:%S')

logging.getLogger().setLevel(logging.ERROR)

# 2. import nessceary library

In [2]:
%pylab inline

import json
import os
import devlib
import trappy
from trace import Trace
from env import TestEnv
from wlgen import RTA, Ramp, Step, Pulse, Periodic

/home/lubaoquan/anaconda2/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


Populating the interactive namespace from numpy and matplotlib


# 3. Configure test env and target

In [4]:
my_target_conf={
    "platform"  : 'android',
    "board"     : 'hikey',
    "device"    : '0123456789',
    "modules"   : [
        'cpufreq'
    ]
}

my_tests_conf={
    "tools"     : ['rt-app', 'taskset', 'trace-cmd'],
    "modules"   : [ "cpufreq" ],
    "ftrace"    : {
        "events"  : [
            "sched_switch",
            "sched_wakeup",
            "sched_wakeup_new",
            "sched_contrib_scale_f",
            "sched_load_avg_cpu",
            "sched_load_avg_task",
            "sched_tune_config",
            "sched_tune_tasks_update",
            "sched_tune_boostgroup_update",
            "sched_tune_filter",
            "sched_boost_cpu",
            "sched_boost_task",
            "sched_energy_diff",
            "cpu_frequency",
            "cpu_capacity",
            "cpu_idle",
        ],
        "buffsize": 10240
    }
}


te = TestEnv(target_conf=my_target_conf, test_conf=my_tests_conf)

target=te.target

# 4. Configure workload

In [ ]:
rtapp=RTA(target, 'simple')
heavy=Periodic(duty_cycle_pct=20, duration_s=30, period_ms=5000)
light=Periodic(duty_cycle_pct=10, duration_s=30, period_ms=5000)

rtapp.conf(
    kind='profile',
    params={
        'task010': light.get(),
        'task01': heavy.get(),
    },
    run_dir=target.working_directory
)

# 5. Run workload and fetch trace

In [ ]:
te.ftrace.start()
rtapp.run(out_dir=te.res_dir)
te.ftrace.stop()
trace_file=os.path.join(te.res_dir, 'trace.dat')
te.ftrace.get_trace(trace_file)
(plt, plt_file)=te.platform_dump(te.res_dir)

# 6. Use trappy to analyze trace

In [ ]:
events_to_parse = [
    "sched_switch",
    "sched_wakeup",
    "sched_wakeup_new",
    "sched_contrib_scale_f",
    "sched_load_avg_cpu",
    "sched_load_avg_task",
    "sched_tune_config",
    "sched_tune_tasks_update",
    "sched_tune_boostgroup_update",
    "sched_tune_filter",
    "sched_boost_cpu",
    "sched_boost_task",
    "sched_energy_diff",
    "cpu_frequency",
    "cpu_capacity",
    "cpu_idle",
]

ftrace=trappy.FTrace(trace_file, normalize_time=True, events=events_to_parse, window=(0, None))
trappy.plotter.plot_trace(ftrace)

# 7. Use Trace to analyze trace

In [ ]:
platformfile=os.path.join(te.res_dir, 'platform.json')

with open(platformfile, 'r') as fh:
    platform = json.load(fh)
    
platform['nrg_model']['little']['cpu']['cap_max'] = 1024
platform['clusters']['little'] = [0, 1, 2, 3]
        
platform['nrg_model']['big']['cpu']['cap_max'] = 1024
platform['clusters']['big'] = [4, 5, 6, 7]
platform['cpus_count'] = 8

logging.info("CPUs max capacities:")
logging.info("   big: %5d (cpus: %s)",
             platform['nrg_model']['big']['cpu']['cap_max'],
             platform['clusters']['big'])
logging.info("LITTLE: %5d (cpus: %s)",
             platform['nrg_model']['little']['cpu']['cap_max'],
             platform['clusters']['little'])

(t_min, t_max)=(0, None)
trace=Trace(platform, te.res_dir, events_to_parse, trace_format="FTrace", window=(t_min, t_max))

trace.analysis.frequency.plotClusterFrequencies()
#trace.analysis.tasks.plotBigTasks(max_tasks=10, min_samples=100, min_utilization=10)
#trace.analysis.tasks.plotWakeupTasks(max_tasks=10, min_wakeups=0, per_cluster=False)
#trace.analysis.status.plotOverutilized()
#trace.analysis.idle.plotClusterIdleStateResidency(clusters=0)
#trace.analysis.idle.plotCPUIdleStateResidency(cpus=1)